### Procesamiento de texto - normalización

In [37]:
def elimina_tildes(cadena):
    """Eliminar acentos de las palabras"""
    a,b = 'áéíóúüÁÉÍÓÚÜ','aeiouuAEIOUU'
    trans = str.maketrans(a,b)
    return(cadena.translate(trans))

def remove_punctuation(text):
    import re,string
    return re.sub('[%s]' % re.escape(string.punctuation), '', text)

In [38]:
def extract_prices(lista):
    "Dada una lista con precios, returna la cantidad como int y su moneda USD o MXN"
    prices=[]
    monedas=[]
    for element in lista:
        if '$' in element:
            prices.append(element)
            """ Checar si son MXN o USD"""
            if 'MXN' in element:
                moneda = 'MXN'
            elif 'USD' in element:
                moneda = 'USD'
            else:
                moneda = 'MXN'
            monedas.append(moneda)
    
    for i in range (len(prices)):
        res = [i for i in prices[i] if i.isdigit()]
        res = ''.join(res)
        prices[i]=int(res)
        
    return(prices, monedas)

In [39]:
def normalizar(input_str):
    "Normaliza el texto -minusculas,puntuación,tildes"
    input_str = input_str.strip() # Remover espacios en blanco en los extremos
    input_str = input_str.lower() # Minusculas
    input_str = elimina_tildes(input_str) # Quitar acentos
    input_str = remove_punctuation(input_str) # Puntuacion
    return(input_str)

In [46]:
def tokenizar(sentence):
    from nltk.corpus import stopwords 
    from nltk.tokenize import word_tokenize 

    sentence = "Hola!"

    stop_words = set(stopwords.words('spanish')) 
    word_tokens = word_tokenize(sentence)

    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
    return(filtered_sentence)

In [9]:
def get_text(url, clase):
    
    import requests
    from bs4 import BeautifulSoup
    
    lista = []
    for i in range(len(url)):
        page = requests.get(url[i])
        soup = BeautifulSoup(page.content)
        soup_class = soup.find_all(class_=clase)

        for item in soup_class:
            lista.append(item.text)
    
    return lista

In [8]:
def get_df(lista):
    aux = [info.strip() for info in lista]
    for i in range(len(aux)):
        aux[i]=aux[i].split('\n')
        aux[i]=[info.strip() for info in aux[i] if info]
    return (aux)

In [10]:
import requests
import lxml.html as lh
import pandas as pd

In [44]:
url='https://www.metroscubicos.com/articulo/consejos/2016/05/18/merida-con-mas-de-30-proyectos-inmobiliarios-nuevos'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [45]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [46]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"
PROYECTO
"
2:"
COLONIA
"
3:"
SUPERFICIE
"
4:"
RECÁMARAS
"
5:"
 $ DESDE
"


In [47]:
len(tr_elements)

14

In [48]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    #if len(T)!=8:
        #break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [49]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [50]:
df.head()

,PROYECTO,COLONIA,SUPERFICIE,RECÁMARAS,$ DESDE
0,\nAltabrisa Residencial\n,\nAltabriza\n,\n222-256 m2\n,3,"\n$ 2’150,000\n"
1,\nGran Santa Fe Norte\n,\nCaucel\n,\n78-231m2\n,\n2 a 3\n,"\n$830,000\n"
2,\nReal Montejo\n,\nReal Montejo\n,\n166.50 m²\n,\n3 a 4\n,"\n$1’411,720\n"
3,\nAlzáre Real Montejo\n,\nReal Montejo\n,\n163 - 163 m2\n,3,"\n$1’385,000\n"
4,\nReal de Dzitia\n,\nReal Montejo\n,\n141 m2\n,3,"\n$1’195,000\n"
